In [ ]:
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import util


In [2]:
df = pd.read_csv("data_step1.csv", low_memory=False)

### drop id and product description, not needed any more

In [3]:
df = df.drop(df.columns[[0, 1]], axis=1)

### Delete wrong data in each column using the clusters 

In [142]:
###check each column clusters to see valid data
for i in range(2,5):
    column_i = df.columns[i]
    column_i_1228 = df.columns[i + 1228]

    grouped = df.groupby(column_i_1228)[column_i].agg(list)

    # Display the result for each cluster
    print(f"Column Number: {df.columns.get_loc(column_i)}\n Values for {column_i} grouped by {column_i_1228}:")
    print(grouped)
    print("\n")


Column Number: 2
 Values for دارای مبدل grouped by دارای مبدل_cluster_labels:
دارای مبدل_cluster_labels
0    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
1    [('استرالیا و چین', 'اروپا', 'آمریکا و ژاپن'),...
2    [('آمریکا و ژاپن',), ('اروپا',), ('آمریکا و ژا...
Name: دارای مبدل, dtype: object


Column Number: 3
 Values for مدل grouped by مدل_cluster_labels:
مدل_cluster_labels
-1    [E5-2640 v2, E5-2695 v2, E5-2670 V2, E5-2680 v...
 0    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
 1                  [Ivy Bridge, Haswell , Surface Duo]
 2                                [شارژی, هشت عدد, چوب]
Name: مدل, dtype: object


Column Number: 4
 Values for هشدار ها grouped by هشدار ها_cluster_labels:
هشدار ها_cluster_labels
-1    [پشتیبانی از شبکههای GPS: A-GPS/GLONASS/Galile...
 0    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
 1    [قابلیت نمایش اعلانهای مختلف پیامک، تماس، تمام...
 2    [جهت سینک شدن ساعت و گوشی از کد سه بعدی داخل س...
Name: هشدار ها, dtype: object




#### Usnig bert first but not applicable enough

In [143]:
###Find Similarity of cluster'value with column name, using a persian BERT

def text_embedding(text, model, batch_size=32, use_cpu=True):
    try:
        # Encode text and move the tensor to CPU before converting to numpy
        if use_cpu:
            embeddings = model.encode(text, convert_to_tensor=True, batch_size=batch_size, device='cpu').detach().numpy()
        else:
            # Explicitly free GPU memory after encoding
            with torch.no_grad():
                embeddings = model.encode(text, convert_to_tensor=True, batch_size=batch_size).cpu().detach().numpy()
        
        # Use flatten instead of reshape
        embeddings = embeddings.flatten()
        return embeddings
    except Exception as e:
        print(f"Error occurred during encoding: {e}")
        return None

def calculate_similarity(embedding1, embedding2):
    # return cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))[0][0]
    return util.pytorch_cos_sim(embedding1, embedding2)[0][0].item()


def check_semantic_similarity(df, column_number, cluster_labels_column, model, threshold=0.3):

    # Get the column name based on the column number
    column_name = df.columns[column_number]
    column_embedding = text_embedding(column_name, model)

    # Get cluster labels from the specified column
    cluster_labels = df.iloc[:, cluster_labels_column]

    # Group by the cluster labels
    grouped = df.groupby(cluster_labels)[column_name].agg(list)

    # Iterate over clusters
    for target_cluster, target_cluster_list in grouped.items():
        # Check if the cluster label is not -1
        if target_cluster != -1:
            # Check if there is data in the list
            if target_cluster_list:
                # Get the first value in the list
                first_value = str(target_cluster_list[0])  # Convert to string to handle numerical values

                # Check if the first value is not nan
                if first_value.lower() != 'nan':
                    # Calculate text embeddings using SentenceTransformer
                    # column_embedding = text_embedding(column_name, model)
                    first_value_embedding = text_embedding(first_value, model)

                    # Check if embeddings were successfully obtained
                    if column_embedding is not None and first_value_embedding is not None:
                        # Calculate cosine similarity
                        similarity_score = calculate_similarity(column_embedding, first_value_embedding)

                        # Check if the similarity score is above the threshold
                        is_related_semantically = similarity_score > threshold

                        # Display the result
                        print(f"Is the first value '{first_value}' in cluster {target_cluster} related to column {column_number} '{column_name}'? {is_related_semantically} (Similarity Score: {similarity_score})")
                else:
                    print(f"The first value in cluster {target_cluster} was 'nan'.")
            else:
                print(f"No data found for cluster {target_cluster}.")
        else:
            print(f"Skipping cluster {target_cluster}.")


In [144]:
### Try different modele and apply
# Load SentenceTransformer model
model_name = '/home/salvia/.cache/torch/sentence_transformers/m3hrdadfi_bert-fa-base-uncased-wikitriplet-mean-tokens'
# model_name = '/home/salvia/.cache/torch/sentence_transformers/sentence-transformers/bert-base-nli-stsb-mean-tokens'
# model_name='sentence-transformers/use-cmlm-multilingual'

# model_name='/home/salvia/.cache/torch/sentence_transformers/HooshvareLab_bert-base-parsbert-uncased'
model = SentenceTransformer(model_name)

for column_number in range(20,25):
    cluster_labels_column = column_number + 1228  # Assuming the cluster labels start from column 1457
    check_semantic_similarity(df, column_number, cluster_labels_column, model, 0.3)
    print("-----------------------")

No sentence-transformers model found with name /home/salvia/.cache/torch/sentence_transformers/m3hrdadfi_bert-fa-base-uncased-wikitriplet-mean-tokens. Creating a new one with MEAN pooling.


Skipping cluster -1.
The first value in cluster 0 was 'nan'.
-----------------------
The first value in cluster 0 was 'nan'.
Is the first value 'شیشه' in cluster 1 related to column 21 'جنس فیلتر'? True (Similarity Score: 0.5647438168525696)
Is the first value 'شیشه AGC' in cluster 2 related to column 21 'جنس فیلتر'? True (Similarity Score: 0.5990363359451294)
-----------------------
Skipping cluster -1.
The first value in cluster 0 was 'nan'.
Is the first value 'Sequans' in cluster 1 related to column 22 'مدل چیپست'? True (Similarity Score: 0.3974950909614563)
Is the first value 'Huawei' in cluster 2 related to column 22 'مدل چیپست'? True (Similarity Score: 0.4286998510360718)
-----------------------
Skipping cluster -1.
The first value in cluster 0 was 'nan'.
Is the first value 'پلاستیک' in cluster 1 related to column 23 'جنس فیس پلیت'? True (Similarity Score: 0.6840038299560547)
Is the first value 'فلزی' in cluster 2 related to column 23 'جنس فیس پلیت'? True (Similarity Score: 0.575

#### Delete the cluster anomali Manually, and save unrelated data to delete them from all the dataset

In [4]:
### Delete the cluster anomali manually, and save unrelated data to delete them from all the dataset
def set_cluster_data_to_nan_multiple(df, column_indices, cluster_numbers_list):
    updates_dict = dict(zip(column_indices, cluster_numbers_list))
    
    # Create a set to store the data before making it NaN
    data_before_nan = set()

    for update in updates_dict.items():
        column_index, cluster_numbers = update

        for cluster_number in cluster_numbers:
            # Identify rows with the specified cluster in the specified column using iloc
            rows_to_update = df[df.iloc[:, column_index + 1228] == cluster_number].index

            # Store the cell values before making them NaN
            data_before_nan.update(set(df.loc[rows_to_update, df.columns[column_index]].values))

            # Set the values in the specified column to NaN for the identified rows
            df.loc[rows_to_update, df.columns[column_index]] = np.nan

    return df, data_before_nan

column_indices = [2, 3, 5,6,7,10,13,17,18,30,38,40,47,49,52,53,54,61,62,72,78,79,86,89,92,97,98,106,107,108, 1035]
cluster_numbers_list = [[1, 2], [2], [1,2],[1,2],[2],[1,2],[1,2],[1,2],[2],[1,2],[1,2],[1],[1,2],[1,2],[1,2],[1,2],[1,2],[1,2],[1,2],[1,2],[1,2],[1,2],[1,2],[1],[1,2],[1,2],[1,2],[1,2],[1,2],[1,2],[0,1,2],[1,2]]

df, data_before_nan = set_cluster_data_to_nan_multiple(df, column_indices, cluster_numbers_list)

In [5]:
### Delete all anomaly data
def trace_and_set_to_nan(df, data_before_nan):
    # for col in df.columns:
    for col in df.columns[2:1229]:
        for index, cell_value in df[col].items():
            # Check if the cell value is in the set data_before_nan
            if cell_value in data_before_nan:
                df.at[index, col] = np.nan

    return df
    
df = trace_and_set_to_nan(df, data_before_nan)

### Delete full null, and cluster cols not needed any more

In [6]:
#Drop columns from 1231 to the end: All extra columns for clustering value,any more neeaded
df = df.drop(df.columns[1229:], axis=1)

In [7]:
### delete nan columns and nan rows
df = df.dropna(axis=1, how='all')  # Drop NaN columns
df = df.dropna(axis=0, how='all')  # Drop NaN rows

### Save result to csv

In [8]:
df.to_csv('data_step2.csv', index=False)